In [33]:
import numpy as np
import functools as ft
import networkx as nx

In [34]:
def hdv(d):
    return np.random.choice([-1, 1], d)


def zero(d):
    return np.zeros((d,))


def bind(xs):
    return ft.reduce(lambda x, y: x * y, xs)


def bundle(xs):
    return ft.reduce(lambda x, y: x + y, xs)


def p(d):
    return np.random.shuffle(np.eye(d))


def inverse(P):
    return np.linalg.inv(P)


def permute(P, H):
    return P.dot(H)


def cosine_similarity(A, B):
    norm_A = np.linalg.norm(A)
    norm_B = np.linalg.norm(B)

    if norm_A == 0 or norm_B == 0:
        return 0

    return np.dot(A, B) / (norm_A * norm_B)


def similarity(A, B):
    return np.dot(A, B) / len(A)


sim = cosine_similarity

In [35]:
class ItemMemory:
    def __init__(self, vectors=[]):
        self.vectors = vectors

    def addVector(self, label, V):
        self.vectors.append((label, V))

    def getVector(self, label):
        return self.vectors[label]

    def cleanup(self, V):
        return max(self.vectors, key=lambda x: sim(V, x[1]))

In [36]:
def initVertices(graph, d=10000):
    for n in graph:
        graph.nodes[n]["hdv"] = hdv(d)

In [37]:
def initNodeMem(graph, memory):
    for n in graph:
        xs = map(lambda x: graph.nodes[x]["hdv"], list(graph.neighbors(n)))
        graph.nodes[n]["mem"] = bundle(xs)
        memory.addVector(f"mem{n}", graph.nodes[n]["mem"])

In [38]:
def retrain(graph, memory, threshold, iter=15):
    count = 0
    for i in range(iter):
        for n in graph:
            mem = graph.nodes[n]["mem"]
            finish = True
            for neighbor in map(
                lambda x: graph.nodes[x]["hdv"], list(nx.neighbors(graph, n))
            ):
                if sim(mem, neighbor) < threshold:
                    mem = bundle([mem, neighbor])
                    finish = False
                    print("here")

            for non_neighbor in map(
                lambda x: graph.nodes[x]["hdv"], list(nx.non_neighbors(graph, n))
            ):
                if sim(mem, non_neighbor) > threshold:
                    mem = bundle([mem, -non_neighbor])
                    finish = False
                    print("here")

            if finish:
                return

            graph.nodes[n]["mem"] = mem
            memory.addVector(f"mem{n}_{i}", mem)

In [39]:
def initGraph(graph, memory):
    G = []
    for n in graph.nodes():
        G.append(bind([graph.nodes[n]["hdv"], graph.nodes[n]["mem"]]))
    G = bundle(G) / 2
    memory.addVector("G", G)
    return G

In [40]:
def checkEdge(G, A, B, threshold):
    return sim(B, bind([G, A])) > threshold

In [41]:
def nodeMemoryReconstruction(G, xs, iter=15):
    if iter == 0:
        return list(map(lambda H: bind([G, H]), xs))

    mems = nodeMemoryReconstruction(G, xs, iter - 1)

    newMems = []
    b = [bind(x) for x in zip(mems, xs)]
    for i in range(len(xs)):
        bmem = bundle(b[:i] + b[i + 1 :])
        newMems.append(bind([xs[i], bundle([G, -bmem])]))

    return newMems

In [42]:
def graphMemoryReconstruction(G, xs, iter=15):
    raise NotImplementedError
    if iter == 0:
        return (G, zeros(len(G)))

    (Gi, Ni) = graphMemoryReconstruction(G, xs, iter - 1)
    Gii = bundle([Gi, -Ni])

    for i in range(iter):
        print(i)

    return bundle([Gi, -edges])

In [43]:
# visited = dict()
visited = {}


def shortestPath(G, A, B, xs, distance, threshold, memory):
    label, value = memory.cleanup(bind([G, A]))

    if np.array_equiv(A, B):
        return 0, label

    if label in visited and visited[label][0] <= distance:
        return visited[label][1], visited[label][2]

    visited[label] = [distance, 99, "memnill"]

    neighbours = list(filter(lambda x: checkEdge(G, A, x, threshold), xs))

    if len(neighbours) == 0:
        return 99, "memnill"

    dis, lab = min(
        list(
            map(
                lambda a: shortestPath(G, a, B, xs, distance + 1, threshold, memory),
                neighbours,
            )
        ),
        key=lambda x: x[0],
    )
    visited[label][1] = dis + 1
    visited[label][2] = label + lab
    return dis + 1, label + lab

---


# Tests


In [44]:
def testCheckEdge(G, graph, threshold):
    count = 0

    for n in graph.nodes():
        for m in graph.nodes():
            if n == m:
                continue
            exist = graph.has_edge(n, m)
            check = checkEdge(
                G, graph.nodes[n]["hdv"], graph.nodes[m]["hdv"], threshold
            )
            if exist != check:
                count += 1
                print(n, m, exist, check)

    print(count, "%.5f" % round(count / EDGES, 5))

In [45]:
def testNodeMemoryReconstruction(G, graph, iter=15):
    for i in range(iter):
        memsi = nodeMemoryReconstruction(
            G, list(map(lambda x: graph.nodes[x]["hdv"], graph.nodes())), iter=i
        )
        print(
            f"{0}_{i:02} =>",
            "%.10f" % abs(sim(graph.nodes[0]["mem"], memsi[0])),
        )

In [46]:
def testShortestPath():
    mem = ItemMemory()

    g = nx.gnm_random_graph(NODES, EDGES)
    initVertices(g, DIMENSIONS)
    initNodeMem(g, mem)
    G = initGraph(g, mem)

    for n in g.nodes():
        print(g.edges(n))

    hdvs = list(map(lambda x: g.nodes[x]["hdv"], g.nodes()))
    origin = 0

    for n in g.nodes():
        print(
            f"{n:02} =>",
            "T" if mem.cleanup(bind([G, hdvs[n]]))[0] == f"mem{n}" else "F",
            mem.cleanup(bind([G, hdvs[n]]))[0],
        )

    testCheckEdge(G, g, THRESHOLD)

    for m in g.nodes():
        if origin == m:
            continue
        path = nx.shortest_path(g, origin, m)
        oracle = len(path) - 1
        visited = {}
        test = shortestPath(G, hdvs[origin], hdvs[m], hdvs, 0, THRESHOLD, mem)
        testPath = list(map(int, test[1].split("mem")[1:]))
        print(
            f"{origin}_{m:02} =>",
            oracle,
            test[0],
            "T  " if oracle == test[0] else "F X",
            path,
            testPath,
            nx.is_path(g, testPath),
        )

---


# Main


In [47]:
NODES, EDGES = 30, 150
DIMENSIONS, THRESHOLD, ITER = 10000, 0.047, 4

In [48]:
def main():
    graph = nx.gnm_random_graph(NODES, EDGES)
    memory = ItemMemory()

    initVertices(graph, DIMENSIONS)
    initNodeMem(graph, memory)
    retrain(graph, memory, THRESHOLD, ITER)
    G = initGraph(graph, memory)
    print("G =>", G)
    # testNodeMemoryReconstruction(G, graph, 1)
    # testCheckEdge(G, graph, THRESHOLD)


# main()
testShortestPath()

[(0, 26), (0, 6), (0, 9), (0, 11), (0, 12), (0, 8), (0, 4), (0, 5), (0, 25), (0, 24), (0, 22)]
[(1, 5), (1, 26), (1, 4), (1, 10), (1, 25), (1, 23), (1, 24), (1, 9), (1, 7), (1, 12), (1, 22), (1, 27)]
[(2, 13), (2, 28), (2, 25), (2, 8), (2, 27), (2, 15), (2, 17), (2, 12), (2, 20), (2, 18), (2, 22), (2, 9)]
[(3, 10), (3, 27), (3, 23), (3, 4), (3, 7), (3, 15), (3, 21), (3, 5), (3, 25), (3, 12)]
[(4, 1), (4, 24), (4, 17), (4, 3), (4, 16), (4, 23), (4, 0), (4, 18)]
[(5, 11), (5, 1), (5, 27), (5, 13), (5, 17), (5, 14), (5, 15), (5, 3), (5, 26), (5, 0), (5, 28), (5, 9)]
[(6, 15), (6, 20), (6, 0), (6, 24), (6, 7), (6, 11), (6, 28), (6, 13), (6, 9)]
[(7, 13), (7, 29), (7, 24), (7, 6), (7, 25), (7, 3), (7, 1), (7, 19)]
[(8, 15), (8, 17), (8, 9), (8, 2), (8, 20), (8, 10), (8, 0)]
[(9, 29), (9, 8), (9, 0), (9, 18), (9, 24), (9, 1), (9, 12), (9, 14), (9, 11), (9, 6), (9, 2), (9, 5)]
[(10, 3), (10, 20), (10, 23), (10, 1), (10, 14), (10, 24), (10, 8), (10, 17), (10, 11), (10, 27), (10, 12), (10, 15),

0_01 => 2 2 T   [0, 5, 1] [0, 4, 1] True
0_02 => 2 2 T   [0, 25, 2] [0, 4, 1] True
0_03 => 2 2 T   [0, 4, 3] [0, 4, 1] True
0_04 => 1 2 F X [0, 4] [0, 4, 1] True
0_05 => 1 2 F X [0, 5] [0, 4, 1] True
0_06 => 1 2 F X [0, 6] [0, 4, 1] True
0_07 => 2 2 T   [0, 24, 7] [0, 4, 1] True
0_08 => 1 2 F X [0, 8] [0, 4, 1] True
0_09 => 1 2 F X [0, 9] [0, 4, 1] True
0_10 => 2 2 T   [0, 24, 10] [0, 4, 1] True
0_11 => 1 2 F X [0, 11] [0, 4, 1] True
0_12 => 1 2 F X [0, 12] [0, 4, 1] True
0_13 => 2 2 T   [0, 12, 13] [0, 4, 1] True
0_14 => 2 2 T   [0, 5, 14] [0, 4, 1] True
0_15 => 2 2 T   [0, 6, 15] [0, 4, 1] True
0_16 => 2 2 T   [0, 11, 16] [0, 4, 1] True
0_17 => 2 2 T   [0, 8, 17] [0, 4, 1] True
0_18 => 2 2 T   [0, 11, 18] [0, 4, 1] True
0_19 => 2 2 T   [0, 25, 19] [0, 4, 1] True
0_20 => 2 2 T   [0, 25, 20] [0, 4, 1] True
0_21 => 2 2 T   [0, 22, 21] [0, 4, 1] True
0_22 => 1 2 F X [0, 22] [0, 4, 1] True
0_23 => 2 2 T   [0, 25, 23] [0, 4, 1] True
0_24 => 1 2 F X [0, 24] [0, 4, 1] True
0_25 => 1 2 F X [0

---
